In [17]:
import random

In [18]:
class Card:
    # The UNO card class
    def __init__(self, color, value):
        self.color = color # Red, Blue, Green, Yellow, Wild
        self.value = value # 0-9, Draw Two, Skip, Reverse, Wild, Wild Draw Four,

    def __str__(self): # Returns a string representation of the card
        return f"{self.color} {self.value}"


In [19]:
class Deck:
    # The UNO deck class
    def __init__(self):
        self.cards = [] # The deck of cards
        self.build() # Builds the deck

    def build(self):
        # Builds the deck
        
        for color in ["Red", "Blue", "Green", "Yellow"]:
            for value in range(0, 10):
                self.cards.append(Card(color, value))  
                if value != 0:
                    self.cards.append(Card(color, value)) # Draw two of each num card except 0
            for value in ["Draw Two", "Skip", "Reverse"]:  # Draw two of each action card
                self.cards.append(Card(color, value))
                self.cards.append(Card(color, value))

        for value in ["Wild", "Draw Four"]:
            for _ in range(4): # Draw four of each wild card
                self.cards.append(Card("Wild", value))
            

    def show(self):
        # Shows the deck
        for card in self.cards:
            print(card)

    def shuffle(self):
        # Shuffles the deck
        for i in range(len(self.cards) - 1, 0, -1):
            r = random.randint(0, i)
            self.cards[i], self.cards[r] = self.cards[r], self.cards[i]

    def draw(self):
        # Draws a card
        #Return none if no cards left
        if len(self.cards) == 0:
            return None

        return self.cards.pop()

In [20]:
class Player:
    # The UNO player class
    def __init__(self, name):
        self.name = name # The player's name
        self.hand = [] # The player's hand

    def draw(self, deck):
        # Draws a card
        newCard = deck.draw() 
        if newCard != None:
            self.hand.append(newCard)
            return True
        else:
            return False
        

    def showHand(self):
        # Shows the player's hand
        print(f"{self.name}'s hand:")
        for i, card in enumerate(self.hand):
            print(f"{i + 1}: {card}")


    def playCard(self, card):
        # Plays a card
        self.hand.remove(card)

    def hasWon(self):
        # Checks if the player has won
        return len(self.hand) == 0

In [21]:
class Game:
    # The Game Class
    def __init__ (self): 
        self.deck = Deck()
        self.currentCard = None
        self.players = []

    def checkCard(self, card):
        # Checks if the card is valid
        if self.currentCard == None:
            return True
        
        # Set the color if the card is wild
        if card.color == "Wild":
            notValidColor = True
            validColors = ["Red", "Blue", "Green", "Yellow"]
            while notValidColor:
                print("What color would you like to change it to?")
                newColor = input(">> ").capitalize()
                if newColor in validColors:
                    card.color = newColor
                    notValidColor = False
                else:
                    print("Invalid color.")

            # Power up cards
            if card.value == "Spy Card": 
                print("You can now see everyone's hand.")
                for player in self.players:
                    player.showHand()

                print("Press enter to continue.")
                input()

            return True
        if card.color == self.currentCard.color or card.value == self.currentCard.value:
            return True
        return False
    
    def start(self):
        self.deck.shuffle()
        # Get Players
        i = 0 # Player number
        while True:
            name = input(f"Enter Player {i + 1}'s name (or leave blank to start game): ")
            if name == "":
                #Verify that there are at least 2 players
                if len(self.players) < 2:
                    print("You must have at least 2 players to play.")
                    continue
                else:
                    break
            self.players.append(Player(name))
            i += 1

            #Break if there are 10 players
            if i == 10:
                print("You can only have up to 10 players... starting game.")
                break

        # Deal 7 cards to each player
        for player in self.players:
            for _ in range(7):
                player.draw(self.deck)

        # Testing: Give a player a spy card
        self.players[0].hand.append(Card("Wild", "Spy Card"))

        # Draw a card from the deck to start the game
        while True:
            self.currentCard = self.deck.draw()
            if self.currentCard.color != "Wild":
                break

        # Round Logic
        currentPlayer = 0
        direction = 1

        while True: #Loop until someone wins
            # Clear the console
            print("\n" * 100)
            print(f"\n{self.players[currentPlayer].name}'s turn.")
            print(f"Current Card: {self.currentCard}") 
            print("Type a number to play a card, type 'draw' to draw a card, 'show' to show your hand, or 'quit' to quit.")

            inputValid = False
            drewCard = False

            while not inputValid:
                # Get input
                playerInput = input(">> ").lower()

                # Play a card
                if playerInput.isdigit():
                    playerInput = int(playerInput) - 1
                    if playerInput < len(self.players[currentPlayer].hand):
                        card = self.players[currentPlayer].hand[playerInput]
                        if self.checkCard(card):
                            self.players[currentPlayer].playCard(card)
                            self.currentCard = card
                            inputValid = True
                            drewCard = True
                        else:
                            print("That card is not valid.")
                    else:
                        print("That card does not exist.")
                # Draw a card
                elif playerInput == "draw":
                    self.players[currentPlayer].draw(self.deck)
                    #Check if the player can play the card they drew
                    if self.checkCard(self.players[currentPlayer].hand[-1]):
                        self.currentCard = self.players[currentPlayer].hand[-1]
                        self.players[currentPlayer].playCard(self.currentCard)
                        drewCard = True

                    inputValid = True
                # Show hand
                elif playerInput == "show":
                    self.players[currentPlayer].showHand()
                # Quit
                elif playerInput == "quit":
                    print("Thanks for playing!")
                    return
                else:
                    print("Invalid input.")

            # Check if the player has won
            if self.players[currentPlayer].hasWon():
                print(f"\n{self.players[currentPlayer].name} has won!")
                break

            if drewCard:
                # Reverse
                if self.currentCard.value == "Reverse":
                    direction *= -1
                    currentPlayer += 1 * direction

                # Skip
                elif self.currentCard.value == "Skip":
                    currentPlayer += 2 * direction

                # Draw Two
                elif self.currentCard.value == "Draw Two":
                    currentPlayer += 1 * direction
                    for _ in range(2):
                        self.players[currentPlayer].draw(self.deck)
                    currentPlayer += 1 * direction

                # Draw Four
                elif self.currentCard.value == "Draw Four":
                    currentPlayer += 1 * direction
                    for _ in range(4):
                        self.players[currentPlayer].draw(self.deck)
                    currentPlayer += 1 * direction

                else:
                    currentPlayer += 1 * direction
            else: 
                currentPlayer += 1 * direction

            # Put the next player in bounds
            currentPlayer = currentPlayer % len(self.players)


def main():
    game = Game()
    game.start()

if __name__ == "__main__":
    main()








































































































Matthew's turn.
Current Card: Blue 4
Type a number to play a card, type 'draw' to draw a card, 'show' to show your hand, or 'quit' to quit.
Matthew's hand:
1: Yellow 2
2: Green 7
3: Blue 9
4: Blue 6
5: Green 1
6: Yellow 1
7: Green Reverse
8: Wild Spy Card
What color would you like to change it to?
You can now see everyone's hand.
Matthew's hand:
1: Yellow 2
2: Green 7
3: Blue 9
4: Blue 6
5: Green 1
6: Yellow 1
7: Green Reverse
8: Yellow Spy Card
John's hand:
1: Red Skip
2: Yellow 6
3: Wild Wild
4: Green 3
5: Green 5
6: Blue 6
7: Yellow 4
Press enter to continue.






































































































John's turn.
Current Card: Yellow Spy Card
Type a number to play a card, type 'draw' to draw a card, 'show' to show your hand, or 'quit' to quit.
John's hand:
1: Red Skip
2: Yellow 6
3: Wild Wild
4: Green 3
5: Green 5
6: Blue 6